# Web Scraping con Selenium

##### Vamos a ver como acceder a una web creando un bot con la librería Selenium que interactuará con la página


## 1. Instalación de librerías

Tenemos que usar las librerías Selenium y Webdriver_manager. Vamos a instalarlas:

## 2. Importación de las librerías

In [43]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


## 3. Instalación del webdriver

El webdriver es lo que nos va a permitir conectarnos con el navegador. Lo instalamos:

In [44]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

## 4. Abrir web
Abrimos la web en el driver ya podemos navegar y acceder a los elementos de la página

In [45]:
url = "https://www.filmaffinity.com/"
driver.get(url)

## 5. Scrapeo
Empezamos lo interesante:

Podemos acceder a elementos en la página de varias maneras:
- Nombre de la etiqueta
- Atributo: Clase
- Atributo: ID
- Atributo: Name
- Selector: Xpath
- Selector: CSS Selector

Para ello vamos a usar el driver que hemos creado y el By.

##### Beginner Selenium Cheatsheet:
Sacar un elemento:
- element = driver.find_element(by, value)

Sacar varios elementos:
- element = driver.find_elements(by, value)

Sacar atributos:
- attribute = element.--el atributo--
- attribute = element.get_attribute(--el atributo--)

Hacer click:
- element.click()

Teclear:
- element.send_keys()

Gestión de pestañas:
- driver.switch_to.window(driver.window_handles[-1])
- driver.get(url)
- driver.close()

Vemos que hay un popup pidiendo que aceptemos las cookies. ¡Vamos a aceptarlo!

In [46]:
elements_by_tag = driver.find_elements(By.TAG_NAME, 'button')
print(type(elements_by_tag))
for i in elements_by_tag:
    print(i.text)
#elements_by_tag[2].click()

<class 'list'>
socios
MÁS OPCIONES
ACEPTO


### Dos formas de obtener todos los botones

Recorriendo la lista

In [47]:
# Encontrar elementos por clase
element_by_class_name = driver.find_element(By.CLASS_NAME,'css-2tkghh')
print(element_by_class_name.find_element(By.TAG_NAME, 'span').text)
# Encontrar elementos por XPATH
element_by_xpath = driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
print(element_by_xpath.text)

MÁS OPCIONES
ACEPTO


Recorriendo los índices

In [48]:
accept = driver.find_element(By.CLASS_NAME, "css-v43ltw")
#Nos aseguramos de que es el botón que estamos buscando
print("Etiqueta: {}".format(accept.tag_name))
print("Texto de la etiqueta: {}".format(accept.text))
print("Atributo mode: {}".format(accept.get_attribute('mode')))
print("Atributo size: {}".format(accept.get_attribute('size')))

accept.click()

Etiqueta: button
Texto de la etiqueta: ACEPTO
Atributo mode: primary
Atributo size: large


Hacemos la búsqueda de una película

In [49]:
search = driver.find_element(By.TAG_NAME, 'input')
search

<selenium.webdriver.remote.webelement.WebElement (session="be849f720b4e5751aafa143612efc4c0", element="813E10FE6526E859E0785AF733AE56DA_element_28")>

In [50]:
search = driver.find_element(By.ID, 'top-search-input-2')
search

<selenium.webdriver.remote.webelement.WebElement (session="be849f720b4e5751aafa143612efc4c0", element="813E10FE6526E859E0785AF733AE56DA_element_28")>

In [51]:
search.send_keys("Oblivion")

In [52]:
search.clear()

In [54]:
def imprimir_poco_a_poco(texto, velocidad=0.15):
    for caracter in texto:
        search.send_keys(caracter)  
        time.sleep(velocidad)  # Pausa entre caracteres

In [55]:
#Introducimos el nombre de la película a buscar
imprimir_poco_a_poco('Oblivion')


search.send_keys(Keys.ENTER)

Entramos en el primer resultado

In [56]:
movie = driver.find_elements(By.CLASS_NAME, "se-it")[0]
movie = driver.find_element(By.CLASS_NAME, "se-it")

#'div.mc-title a'
#'div.mc-info-container a'

#Accedemos mediante un selector CSS al elemento a, que contiene el enlace (texto)
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-title a')

#Accedemos mediante un selector CSS al elemento a, que contiene el enlace (texto) desde el padre
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-info-container a')

In [70]:
#Accedemos mediante la clase al elemento a, que contiene el enlace (texto)
url_titulo = movie.find_element(By.CLASS_NAME, 'mc-title')
url=url_titulo.find_element(By.TAG_NAME, 'a')
url.text


'Oblivion'

In [79]:
url = movie.find_element(By.CSS_SELECTOR, 'div.mc-poster a')



In [62]:
movie.screenshot('pelicula.png')

True

https://www.filmaffinity.com/es/film618375.html


In [ ]:
#Volver
#driver.back()

Vamos a sacar los datos principales de la película:

Título original - Oblivion
Año - 2013
Duración - 126 min.
País -  Estados Unidos
Dirección - Joseph Kosinski
Guion - Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson
Reparto - Tom Cruise
Andrea Riseborough
Olga Kurylenko
Morgan Freeman
Nikolaj Coster-Waldau
Zoe Bell
Música - Anthony Gonzalez, M83, Joseph Trapanese
Fotografía - Claudio Miranda
Compañías - Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios
Género - Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic
Sinopsis - Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Drones que participa en una operación para extraer los recursos vitales del planeta. Su misión consiste en patrullar diariamente los cielos. Un día, resc

In [24]:
movie_dict

{'Título original': 'Oblivion',
 'Año': '2013',
 'Duración': '126 min.',
 'País': ' Estados Unidos',
 'Dirección': 'Joseph Kosinski',
 'Guion': 'Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson',
 'Reparto': 'Tom Cruise\nAndrea Riseborough\nOlga Kurylenko\nMorgan Freeman\nNikolaj Coster-Waldau\nZoe Bell',
 'Música': 'Anthony Gonzalez, M83, Joseph Trapanese',
 'Fotografía': 'Claudio Miranda',
 'Compañías': 'Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios',
 'Género': 'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic',
 'Sinopsis': 'Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Drones que participa en una operación para extraer los recursos vitales del planeta. Su mi

#### Creamos un dataframe

In [25]:
df= pd.DataFrame([movie_dict]) #es decir, mediante una lista de diccionarios (1 diccionario)


In [26]:
df

,Título original,Año,Duración,País,Dirección,Guion,Reparto,Música,Fotografía,Compañías,Género,Sinopsis
0,Oblivion,2013,126 min.,Estados Unidos,Joseph Kosinski,"Joseph Kosinski, Michael Arndt, Karl Gajdusek....",Tom Cruise\nAndrea Riseborough\nOlga Kurylenko...,"Anthony Gonzalez, M83, Joseph Trapanese",Claudio Miranda,"Universal Pictures, Chernin Entertainment, Rel...",Ciencia ficción. Intriga | Futuro postapocalíp...,Año 2073. Hace más de 60 años la Tierra fue at...


#### EXTRA

Abrir nueva ventana:

In [27]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [28]:
driver.switch_to.window(driver.window_handles[-1])

Cerrar ventana

In [29]:
driver.close()